# 분류 - 영화추천

## #01. 패키지

In [11]:
# 경고 메시지 출력 방지
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
from pandas import read_csv, DataFrame, pivot_table
from matplotlib import pyplot as plt
import seaborn as sb

# 분류 알고리즘 관련 패키지
from sklearn.neighbors import KNeighborsClassifier
# 분류 결과 평가를 위한 패키지
from sklearn.metrics import accuracy_score

# 교차검증 패키지
from sklearn.model_selection import cross_val_score

## #02. 데이터 가져오기

### jupyter가 참조하고 있는 현재 디렉토리 확인

In [2]:
print(os.getcwd())

c:\Users\jehyu\Desktop\nationalSupport\jupyter\F. 데이터 마이닝\02. Sklearn


### 영화 데이터 가져오기

분석 후 결과값을 맵핑하기 위한 데이터

In [3]:
origin_mv = read_csv("C:\\Users\\jehyu\\Desktop\\nationalSupport\\netflix\\Netflix_Dataset_Movie.csv", encoding='utf-8')
origin_mv.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


### 별점 데이터 가져오기

In [4]:
origin_rating = read_csv("C:\\Users\\jehyu\\Desktop\\nationalSupport\\netflix\\Netflix_Dataset_Rating.csv", encoding='utf-8')
origin_rating.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


## #03. 데이터 전처리

### 별점 데이터 재구조화

각 영화를 컬럼으로, 사용자 번호를 인덱스로 하는 피벗 테이블을 구성.
> 시간이 걸림

In [5]:
movie_users = pivot_table(origin_rating, index="Movie_ID", columns="User_ID", values="Rating")
movie_users.head()

User_ID,6,7,79,97,134,169,183,188,195,199,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Movie_ID,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 데이터 정제

NaN값은 안본거기 때문에 별점을 0으로 대체

In [6]:
movie_users.isna().sum()

User_ID
6          1205
7          1158
79         1161
97         1238
134        1184
           ... 
2649370    1288
2649378    1281
2649388    1274
2649426    1276
2649429    1288
Length: 143458, dtype: int64

In [7]:
movie_users.fillna(0, inplace=True)
movie_users.head()

User_ID,6,7,79,97,134,169,183,188,195,199,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Movie_ID,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
movie_users.isna().sum()

User_ID
6          0
7          0
79         0
97         0
134        0
          ..
2649370    0
2649378    0
2649388    0
2649426    0
2649429    0
Length: 143458, dtype: int64

### 독립변수(feature), 종속변수(label) 분리

$x$는 데이터프레임 자체

$y$는 데이터프레임 인덱스

In [9]:
x = movie_users.copy()
y = movie_users.index
x.shape, y.shape

((1350, 143458), (1350,))

## #04. 분류 모델 구축

### 단일 수행

In [10]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x, y)
y_pred = knn.predict(x)
score = accuracy_score(y, y_pred)
print("분류 정확도 : {:.2f}%".format(score))

분류 정확도 : 0.17%


### 최적의 k 찾기

In [ ]:
k_range = range(1, len(x.columns))
k_scores = []

for k in k_range:   #knn 컬럼 수만큼 진행
    knn = KNeighborsClassifier(n_neighbors=k)
    # score = cross_val_score(knn, x, y, cv=5).mean()    #knn 5번 수행
    knn.fit(x,y)
    y_pred = knn.predict(x)
    score = accuracy_score(y, y_pred)

    if np.isnan(score):
        break
    k_scores.append(score)
k_scores